## 必要なライブラリのimport(Notebookを開いたり, Kernel再起動の度に一度だけ実行)

In [1]:
import pandas as pd
import numpy as np
import copy
import time
import rclpy
from rclpy.node import Node
from scipy.spatial.transform import Rotation as R
from geometry_msgs.msg import TransformStamped
from std_msgs.msg import Empty
from triorb_drive_interface.msg import TriorbSetPos3 as TriorbSetPos3Topic
from triorb_drive_interface.msg import TriorbRunResult
from triorb_drive_interface.msg import DriveGains
rclpy.init()

## 事前定義(Notebookを開いたり, Kernel再起動の度に一度だけ実行)

In [ ]:
def quaternion_to_euler_zyx(q):
    r = R.from_quat([q[0], q[1], q[2], q[3]])
    return r.as_euler('zyx', degrees=False)

def get_topic_name(topic_name_raw):
    return (os.getenv('ROS_PREFIX', '')+'/'+topic_name_raw).replace("//","/")

## 経由地点リストの初期化

In [2]:
waypoint_list = pd.DataFrame(index=[], columns=["x","y","deg","v_xy","v_w","tx","ty","tr"])

## 経由地点の登録
### "リモコンでロボットを経由地点まで動かし下のCellを実行"を経由地点の数に応じて繰り返す

In [ ]:

class SampleNodeClient(Node):
    def __init__(self):
        super().__init__('sample_node_notebook')
        self.timer = self.create_timer(5.0, self.callback_timer)
        self.subs = [self.create_subscription(TransformStamped, get_topic_name('/vslam/rig_tf'), self.callback_rig_tf, 10)]

    def destroy(self):
        _ = [self.destroy_subscription(_sub) for _sub in self.subs]
        self.destroy_node()
    
    def callback_timer(self):
        print("waiting for the massage")

    def callback_rig_tf(self, msg):
        global waypoint_list
        print(str(msg))
        _x, _y = msg.transform.translation.x, msg.transform.translation.y
        _yaw, _pitch, _roll  = quaternion_to_euler_zyx([msg.transform.rotation.x, msg.transform.rotation.y, msg.transform.rotation.z, msg.transform.rotation.w])
        _df = pd.DataFrame([[_x, _y, -np.rad2deg(_yaw), 0.1, 3.14/2.0, 0.005, 0.005, 0.5, 1.0]], columns=["x","y","deg","v_xy","v_w","tx","ty","tr","sleep"])
        #_df = pd.DataFrame([[_x, _y, -np.rad2deg(_yaw), 0.5, 3.14, 0.05, 0.05, 1.0, 0.0]], columns=["x","y","deg","v_xy","v_w","tx","ty","tr","sleep"])

        waypoint_list = pd.concat([waypoint_list, _df]).reset_index(drop=True)
        raise KeyboardInterrupt
        
node = SampleNodeClient()
try:
    rclpy.spin(node)
except KeyboardInterrupt:
    print("KeyboardInterrupt")
node.destroy()
if waypoint_list.shape[0]:
    print(waypoint_list)

## 速度や目標精度を手で修正（必要に応じて下のセルを実行）

In [ ]:
# 経由地点index 0までの水平移動速度を0.3m/sに設定する
waypoint_list.loc[0, 'v_xy'] = 0.3
# 経由地点index 0までの旋回速度を1.57rad/sに設定する
waypoint_list.loc[0, 'v_w'] = 1.57
# 経由地点index 0の目標位置精度を±0.05mに設定する
waypoint_list.loc[0, ['tx','ty']] = 0.05
# 経由地点index 0の目標角度精度を±1degに設定する
waypoint_list.loc[0, 'tr'] = 1.0
# 経由地点index 0で1秒停止するように設定する
waypoint_list.loc[0, 'sleep'] = 1.0
# 経由地点一覧を表示
print(waypoint_list)

## 自律移動を実行する

In [ ]:
STD_ACC = 1000
STD_DEC = 1000
GAIN_XY_P = 0.22 # 0.2
GAIN_XY_I = 0.0
GAIN_XY_D = 1e-4 # -5
GAIN_W_P = 0.20
GAIN_W_I = 0.0
GAIN_W_D = 1e-5 # -6

# 経由地点一覧を表示
print(waypoint_list)

class SampleNavigateClient(Node):
    def __init__(self, waypoints):
        super().__init__('sample_navigate_notebook')
        self.pub_gain = self.create_publisher(DriveGains, get_topic_name("/setting/drive/gains"), 1)
        self.pub_set_pos = self.create_publisher(TriorbSetPos3Topic, get_topic_name("/drive/set_pos"), 1)
        self.pub_stop = self.create_publisher(Empty, get_topic_name("/drive/stop"), 10)
        self.subs = [self.create_subscription(TriorbRunResult, get_topic_name('/drive/result'), self.callback_result, 1)]
        self.timer = self.create_timer(0.2, self.timer_pub)
        self.waypoints = copy.deepcopy(waypoints)
        self.waypoints = pd.concat([pd.DataFrame([self.waypoints.iloc[0,:]]), self.waypoints], axis=0).reset_index(drop=True) # 不具合に対する暫定対応
        self.msg = None
        self.sleep_time = 0.0
        

    def timer_pub(self):
        if self.msg is not None:
            #print(str(self.msg))
            self.pub_set_pos.publish(self.msg)
            self.msg = None
            
    def set_gain(self):
        msg = DriveGains()
        msg.xy_p = GAIN_XY_P
        msg.xy_i = GAIN_XY_I
        msg.xy_d = GAIN_XY_D
        msg.w_p = GAIN_W_P
        msg.w_i = GAIN_W_I
        msg.w_d = GAIN_W_D
        print(str(msg))
        self.pub_gain.publish(msg)

    def run_next(self):
        for idx, point in self.waypoints.iterrows():
            time.sleep(self.sleep_time)
            self.sleep_time = point.sleep
            request = TriorbSetPos3Topic()
            request.pos.position.x = point.x
            request.pos.position.y = point.y
            request.pos.position.deg = point.deg
            request.pos.speed.xy = point.v_xy
            request.pos.speed.w = point.v_w
            request.pos.speed.acc = STD_ACC
            request.pos.speed.dec = STD_DEC
            request.setting.tx = point.tx
            request.setting.ty = point.ty
            request.setting.tr = point.tr
            request.setting.force = 1
            print(str(request))
            #self.pub_set_pos.publish(request)
            self.waypoints = self.waypoints.drop(idx, axis=0)
            self.msg = request
            print(str(self.waypoints))
            return
        raise KeyboardInterrupt

    def callback_result(self, msg):
        print("Receive results")
        print(str(msg))
        self.run_next()
    
    def destroy(self):
        self.pub_stop.publish(Empty())
        _ = [self.destroy_subscription(_sub) for _sub in self.subs]
        self.destroy_node()


tmp_list = waypoint_list
#tmp_list = waypoint_list.iloc[::-1]
#tmp_list = pd.concat([tmp_list,tmp_list[::-1]], axis=0)
node = SampleNavigateClient(tmp_list)
for _ in range(10):
    rclpy.spin_once(node)
time.sleep(5)
try:
    node.set_gain()
    node.run_next()
    rclpy.spin(node)
except KeyboardInterrupt:
    print("KeyboardInterrupt")
node.destroy()
print("Finish")

## 登録地点の保存

In [79]:
waypoint_list.to_csv("./waypoint.csv")

## 登録地点の呼び出し

In [ ]:
waypoint_list = pd.read_csv("./waypoint.csv")